# How is babby formed

In [ ]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
from fastai.text import *

In [ ]:
path = untar_data(URLs.YAHOO_ANSWERS)
path.ls()

In [ ]:
train = pd.read_csv(header=None, filepath_or_buffer='/storage/yahoo_answers_csv/train.csv')
train = train.replace(pd.np.nan, '', regex=True)
train['all'] = train[1] + ' ' + train[2] + ' ' + train[3]
train.tail()

In [ ]:
test = pd.read_csv(header=None, filepath_or_buffer='/storage/yahoo_answers_csv/test.csv')
test = test.replace(pd.np.nan, '', regex=True)
test['all'] = test[1] + ' ' + test[2] + ' ' + test[3]
test.tail()

In [ ]:
total_df = pd.concat([test, train]); total_df

In [ ]:
bs=48


In [ ]:
defaults.cpus=4

In [ ]:
data_lm = (TextList.from_df(df=total_df, cols=3)
           #Inputs: all the text files in path
            .split_by_rand_pct(0.1)
           #We randomly split and keep 10% (10,000 reviews) for validation
            .label_for_lm()           
           #We want to do a language model so we label accordingly
            .databunch(bs=bs))
data_lm.save('data_lm.pkl')


In [ ]:
data_lm.show_batch()

In [ ]:
data_lm = load_data(file='data_lm.pkl', path='', bs=bs)

In [ ]:
data_lm.show_batch()

In [ ]:
learn = language_model_learner(data_lm, AWD_LSTM, drop_mult=0.3)

In [ ]:
learn.lr_find()
learn.recorder.plot(skip_end=15)

In [ ]:
import torch
torch.cuda.is_available()

In [ ]:
learn.fit_one_cycle(1, 3e-2, moms=(0.8,0.7))

In [ ]:
learn.save('fit_head')

In [ ]:
learn.load('fit_head');